In [17]:
import cv2
import argparse
import numpy as np
import os
import pickle
from tqdm import tqdm

In [18]:
descpt_id = 'surf'
images_path = '../images/database'
images = os.listdir(images_path)
images = [os.path.join(images_path, item) for item in images]

if descpt_id == 'sift':
    feature_extractor = cv2.xfeatures2d.SIFT_create()
elif descpt_id == 'surf':
    feature_extractor = cv2.xfeatures2d.SURF_create(extended=True)   # For getting descriptor of size 128 instead of 64
else:
    feature_extractor = cv2.ORB_create()

# if not os.path.exists(args.descriptors):
print('[INFO] Creating descriptors...')
descriptors = dict()
for image in tqdm(images, desc='Processing images: '):
    image_gray = cv2.imread(image, 0)
    image_gray = cv2.resize(image_gray,(400,300))
    try:
        keypoint, descriptor = feature_extractor.detectAndCompute(image_gray, None)
    except Exception as e:
        print(e)
        pdb.set_trace()
    image_name = image.split('/')[-1].split('.')[0]
    if image_name not in descriptors.keys():
        descriptors[image_name] = descriptor
    else:
        print('[ERROR] Issue with file {}!\nCheck for duplicate images.'.format(image))
with open(descriptors1, 'wb+') as file:
    pickle.dump(descriptors, file)
# else:
#     print('[INFO] Using previously saved descriptors...')
#     with open(args.descriptors, 'rb') as file:
#         descriptors = pickle.load(file)
if len(descriptors.keys()) != len(images):
    raise ValueError("Number of descriptors do not match with number of images in the dataset folder. Create a new set of descriptors.")
else:
    print("done")

[INFO] Creating descriptors...



Processing images: 100%|██████████| 100/100 [00:24<00:00,  4.05it/s]


done


In [19]:
l = '../pickle'
load = 'Y'
GOOD_MATCH_THRESH = 10

query_images = '..images/query/'
database_images = '../images/database/'
descriptor_file = l
load_descriptor =load

database_descriptors = {}
name_list = []
surf = cv2.xfeatures2d.SURF_create(extended=True)
#feature_extractor = cv2.xfeatures2d.SIFT_create()

if load_descriptor == 'Y' or load_descriptor =='y':
    print("Loading Pre Computed Descriptors")
    assert os.path.exists(descriptor_file)
    with open(descriptor_file, 'rb') as file:
        database_descriptors = pickle.load(file)
else:
    for image_name in tqdm(os.listdir(database_images)):
        img = cv2.imread(database_images+image_name,0)
        keypoint, descriptor = surf.detectAndCompute(img, None)
        img_name,img_extension = os.path.splitext(image_name)
        database_descriptors[img_name] = descriptor
        name_list.append(img_name)
    with open(descriptor_file, 'wb') as file:
        pickle.dump(database_descriptors, file)

print("--Database Descriptors Loaded--")

correct_count = 0


for query_name in tqdm(os.listdir(query_images)):
    max_count_name = None
    max_count = 0
    query = cv2.imread(query_images+query_name, 0)
    _, q_desc = surf.detectAndCompute(query, None)
    for database_name in database_descriptors.keys():
        d_desc = database_descriptors[database_name]
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 100)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(q_desc, d_desc, k=2)
        matches_mask = [[0, 0] for i in range(len(matches))]
        distances = list()
        counts = list()
        for i, (m, n) in enumerate(matches):
            if m.distance < 0.7 * n.distance:
                matches_mask[i] = [1, 0]
                distances.append(m.distance)
        counts.append(len(distances))
        if len(distances) != 0:
            if np.max(counts) > max_count:
                max_count = np.max(counts)
                max_count_name = database_name.split('/')[-1].split('.')[0]
    if query_name.split('/')[-1].split('.')[0] == max_count_name:
        correct_count += 1
    else:
        print('[ERROR] database_image: {}, query_image: {}'.format(max_count_name, query_name.split('/')[-1].split('.')[0]))
print('[INFO] Correct matches: {} total queries: {} Accuracy: {}'.format(correct_count, len(os.listdir(query_images)), correct_count/len(os.listdir(query_images))))


Loading Pre Computed Descriptors



  0%|          | 0/100 [00:00<?, ?it/s]

--Database Descriptors Loaded--



  2%|▏         | 2/100 [00:35<32:45, 20.06s/it]

[ERROR] database_image: New_Belgium_Ranger_Bottle, query_image: hailstorm_brewing


...

[ERROR] database_image: New_Belgium_Slow_Ride, query_image: miller_lite



 99%|█████████▉| 99/100 [45:17<00:31, 31.44s/it]

[ERROR] database_image: North_Coast_Steller_Bottle, query_image: ufo-gingerland



100%|██████████| 100/100 [45:43<00:00, 27.43s/it]

[INFO] Correct matches: 27 total queries: 100 Accuracy: 0.27
